Copyright Optimizing Mind 2023
Optimizing Mind Jupyter notebook for benchmarking transfer learning.
This code compares transfer learning for cats and dogs and is based on the tesor flow tutorial and code.
The code: 
1) initiates the Optimizing Mind API 
2) downloads cats dogs data 
3) runs the tensor flow example learning code
4) runs OM API learnin on the exact same data samples as in 3
5) displays verification and learning curves

Optimizing Mind User Guide for API https://docs.google.com/document/d/1Wc2j1Uq6euhuYCsEbvcC8V5xvr5Z4JBf
Steps to run Optimizing Mind API
1) Register and obtain API token https://om-learn-api.azurewebsites.net/
2) copy your token and place in \<token\>

Changes from original TF tutorial code
1) top layer changed from binary to multiclass to allow for multiple labels (no penalty in performance)
2) batch size is changed from the original 32 to 1 in order to be able to better observe OM learning curve (no penalty in performance)
3) TF training loop code changed to a manual one (for loop instead of fit)
4) options added for different datasets
5) options added for different bases
original code: https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/transfer_learning.ipynb

Note code is slow because lots of validation is run on both models, but it produces detailed learning curves

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import keras
import time
import json
import requests

In [ ]:
def Send_to_API(base_name,dataset_name, top_layer_inputs, labels,num_outputs,verbose=False):
    try:
        top_layer_inputsL = top_layer_inputs.tolist() # inputs
        labelsL = labels.tolist() #labels
        num_outputsL = str(num_outputs) 

    except Exception as e:
        if verbose: print(e)
        return(e) 

    username = os.getenv('JUPYTERHUB_USER')
    payload = {"model_name": "test_model99",'inputs': top_layer_inputsL, 'labels': labelsL,'num_outputs':num_outputsL}
    headers = { 'content-type': 'application/json', 'accept': 'application/json','Authorization':'Bearer <token>' }

    r = requests.post(url = 'https://om-learn-api.azurewebsites.net/api/train/', data = json.dumps(payload), headers=headers)
    result = json.loads(r.text)
    return(result)

In [ ]:
def Get_from_API(data): # within compiled code
    weights = np.array(data["result"]) 
    return(weights)

In [ ]:
from datetime import datetime
now=datetime.now()
date_time=now.strftime("%m-%d-%Y")

In [ ]:
date_time

In [ ]:
print(tf.__version__)

In [ ]:
BATCH_SIZE =1 #32   #1
IMG_SIZE = (160, 160)

In [ ]:
# sets up the directories automatically  
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

In [ ]:
# Datasets to benchmark

dataset_name='cats_and_dogs_filtered'  #originalTF notebook dataset
#dataset_name='Animals_filtered'  # 50 animal dataset
#dataset_name='fowl_data' # from https://learn.microsoft.com/en-us/azure/machine-learning/how-to-train-pytorch

PATH2 = os.path.join(os.path.dirname(path_to_zip),dataset_name )    
print(PATH2)

In [ ]:
train_dir = os.path.join(PATH2, 'train')
validation_dir = os.path.join(PATH2, 'validation')

print(PATH2,'\n',train_dir,'\n',validation_dir)

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                                         shuffle=True,
                                                                         batch_size=BATCH_SIZE,
                                                                         image_size=IMG_SIZE)

validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                              shuffle=True,
                                                                              batch_size=BATCH_SIZE,
                                                                              image_size=IMG_SIZE)

In [ ]:
print("Dataset class names:", train_dataset.class_names)

In [ ]:
class_names = train_dataset.class_names
num_outputs=len(class_names)
print('number of classes and outputs:',num_outputs)

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [ ]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

In [ ]:
print('Number of training batches: %d' % tf.data.experimental.cardinality(train_dataset))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [ ]:
initial_epochs = 1 #10  

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)

In [ ]:
# choose base

MODEL = 'MobileNetV2'
#MODEL = 'ResNet50'


In [ ]:
if MODEL=='MobileNetV2':
    preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
    base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights='imagenet')
elif MODEL=='ResNet50':
    preprocess_input = tf.keras.applications.resnet50.preprocess_input
    base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights='imagenet')
else:
    error("Model Not supported")


In [ ]:
base_model.trainable = False

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

In [ ]:
# the original adds 2d global_average_pooling2d layer to make the base layer ready for classification.  
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
# set up learning and top layer, changed from binary to multiclass so softmax added to top
final_output_layer=tf.keras.layers.Softmax()

In [ ]:
# This is the multiclass equivalent to original transfer learning network  
prediction_layer4 = tf.keras.layers.Dense(num_outputs)  # change1 from original: I make num_outputs outputs
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = prediction_layer4(x)    # adds the 49 nodes
outputs = final_output_layer(x)  # change2: adds softmax
orig_TF_paradigm_model = tf.keras.Model(inputs, outputs)   # this is now the traditional leanring network

In [ ]:
# Change 3:SparseCategoricalCrossentropy from BinaryCrossentropy because it is multiclass

base_learning_rate = 0.0001
orig_TF_paradigm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'],
              run_eagerly=True,) # Now I compile it.  

In [ ]:
orig_TF_paradigm_model.summary()

In [ ]:
# create the version of the base model called model_with_av_layer that will feed inputs to OM layer 
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
outputs = global_average_layer(x)
#outputs = prediction_layer(x)   # It is basically the same as the original notebook but no prediction layer
model_with_av_layer = tf.keras.Model(inputs, outputs) 


In [ ]:
model_with_av_layer.summary()

In [ ]:
num_inputs=model_with_av_layer.output.shape[1]  

In [ ]:
# this is an equivalent TF model that will recieve the OM weights
prediction_layer2 = tf.keras.layers.Dense(num_outputs)  # top nodes
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = prediction_layer2(x)  # nodes whose weights I will be changed by om
outputs = final_output_layer(x)  #softmax
tf_model_tobe_trained_by_OM = tf.keras.Model(inputs, outputs)


In [ ]:
#it needs compile if I do validation but I never really do any learning so most of this doesnt really matter
base_learning_rate = 0.0001
tf_model_tobe_trained_by_OM.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'],
              run_eagerly=True,) # needs to be added to prevent execution optimization (not data optimiziation)

In [ ]:
tf_model_tobe_trained_by_OM.summary()

In [ ]:
# does not validate at every point for graphing (otherwise things take forever)
validate_points=np.concatenate([np.arange(30),np.arange(30,40,2),np.arange(40,60,3),np.arange(60,100,5),np.arange(100,300,7),np.arange(300,700,10),np.arange(700,1500,15),np.arange(1500,3000,20)])

In [ ]:
len(validate_points)

In [ ]:
# find the number of entries in validation set
num_data_in_validation_dataset=0
for data, label in validation_dataset: #.as_numpy_iterator():
    num_data_in_validation_dataset=num_data_in_validation_dataset+len(data)
print(num_data_in_validation_dataset)

In [ ]:
# setup validation manually so I dont have to go through base for every validation (saves time because validation is the same)
i=0
validation_labels=np.ones((1,num_data_in_validation_dataset))*50  #crazy initial values for debug
net_out_validation_for_OM=np.ones((num_data_in_validation_dataset,num_inputs))*50  

for data, label in validation_dataset: #.as_numpy_iterator():
    len_batch=len(label)
    validation_labels[0,(BATCH_SIZE*i):(BATCH_SIZE*(i)+len_batch)]=label  # set up validation truth table
    temp=model_with_av_layer.predict(data)  #run bottom transfer layers pre-rfn
    net_out_validation_for_OM[BATCH_SIZE*i:(BATCH_SIZE*i+len_batch),:]=temp[0:len_batch,:]
    i=i+1
validation_labels=validation_labels.astype(int)


In [ ]:
print(validation_labels[0,:])

In [ ]:
# variables to store performance of batches for both approaches
num_batches_in_train_dataset=len(train_dataset)              # number of batches in train_dataset
print("num of data of train batches in a single epoch",num_batches_in_train_dataset,"batch size =",BATCH_SIZE,"epochs=",initial_epochs)
OM_learn_Vacc=[] 
tf_learn_Vacc=[]

In [ ]:
# store baselines before learn
store_baselines_before_learning=False
if store_baselines_before_learning:
    loss_, accuracy_ = orig_TF_paradigm_model.evaluate(validation_dataset)
    print('validation of TF before train on new',accuracy_)
    tf_learn_Vacc.append(accuracy_) #store data

    OMmodel.test_data_score(net_out_validation_for_OM,validation_labels[0,:])  #validate for OM
    print('validation of OM on NEW data before train on new',OMmodel.last_score_accuracy/100)
    OM_learn_Vacc.append(OMmodel.last_score_accuracy/100) 
    
    # add one more zero point and increase the index of everything by 1 to test before learning
    graph_points=list(validate_points.copy()+1)
    graph_points.insert(0,0)
else:
    graph_points=list(validate_points.copy())


In [ ]:
initial_epochs=1

In [ ]:
verbose=False
labels_presented=[]

In [ ]:
for j in range(initial_epochs):
    if j>0:  #update graph points if doing more than one epoch
        validate_points=np.concatenate([np.arange(0,3000,30)])
        graph_points[num_stored:]=[]
        graph_points=graph_points+list(validate_points.copy()+len(labels_presented)+1)   #need to figure out graphing stuff

    for i, data in train_dataset.enumerate():   
        
        labels_presented.append(data[1])  # store record of labels presented
                
        #the original learning        
        batch_logs = orig_TF_paradigm_model.train_step(data)
                
        # prepare data to send out to OM.        
        net_out = model_with_av_layer.predict_on_batch(data[0])  # run through base layer to have data ready for OM
        
        
        # call to the API
        json_back=Send_to_API(MODEL,dataset_name, net_out, data[1].numpy(),num_outputs, verbose=verbose)
        
        
        outfrom_Get_from_API=Get_from_API(json_back) #extract weights from API results
        
        u_model=np.array(outfrom_Get_from_API).T
        
        
        if verbose: 
            print("input size: ",net_out.shape,"label size: ",data[1].shape)
            print("label is:",np.array(data[1]))
            print("label is:",data[1].numpy())
            print("net_out",net_out)
            print("num_outpus",num_outputs)
            print("Json back in notebook",json_back)
            print("outfrom_Get_from_API",outfrom_Get_from_API)
            print("u_model.shape",u_model.shape)
        
        # put weights that come back into final OM generated transfer learning network: 
        tf_model_tobe_trained_by_OM.trainable_weights[0].assign(tf.Variable(np.float32(u_model)))     #OMmodel.u_model.T))) 
        # this is now the TF equivalent of OM learned net 
        
        # validation code run in specified validation points
        if i in validate_points:  # minimizing number of validations because it takes too long
        
            # validating TF learned top layer
            loss_, accuracy_ = orig_TF_paradigm_model.evaluate(validation_dataset)

            # adding TF accuracies to the record
            tf_learn_Vacc.append(accuracy_)  
            
            # validating OM learned top layer but equivalent net to TF
            loss0, accuracy_OMTF = tf_model_tobe_trained_by_OM.evaluate(validation_dataset)  # it is validated and run just like the original TF

            OM_learn_Vacc.append(accuracy_OMTF)

            print("Training: Batch {}, Epoch {}, {} entries accuracy of OM {} and TF {} ".format(i,j, len(data[0]), np.round(OM_learn_Vacc[-1],4),np.round(accuracy_,4)))
            
    
    num_stored=len(OM_learn_Vacc)
    

In [ ]:
print(len(OM_learn_Vacc))

In [ ]:
# indicate when all animals are presented at least once
all_presented=1
while len(np.unique(labels_presented[0:all_presented])) != len(np.unique(labels_presented)):
    all_presented=all_presented+1
all_presented=np.where(np.array(graph_points) >= all_presented)[0][0]   #first index where all presented
print('first validation run after all animals were presented:',all_presented,'\nwhich begins with training instance',graph_points[all_presented])

In [ ]:
grange=int(graph_points[num_stored]*.02)     #90  #31 #90 #121  # choosing a nice zoom range
if grange==0:
    grange=num_stored
maxpoint=max(graph_points[0:grange])
print(maxpoint)

In [ ]:
grange

In [ ]:
# zoomed figure
print('first',graph_points[grange],'plots')

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)

plt.plot(graph_points[0:(all_presented+1)],OM_learn_Vacc[0:(all_presented+1)],color='darkblue')
plt.plot(graph_points[all_presented:grange],OM_learn_Vacc[all_presented:grange],color='cornflowerblue', label='OM Accuracy NEW')
plt.plot(graph_points[0:grange],tf_learn_Vacc[0:grange],color='orange', label='TF Accuracy NEW')



plt.legend(loc='lower right')
plt.ylabel('Validation Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.ylim([0.4,1.0])
plt.title("OM vs TF {} Accuracy within {} Batch={} Example(s) Trained".format(dataset_name,graph_points[grange],BATCH_SIZE))

plt.xlabel('Batch Number')
plt.show()
print('Labels  ',np.array(labels_presented)[0:grange].T)

In [ ]:
grange=np.int(num_stored*.05)

In [ ]:
print(max(graph_points),num_stored,graph_points[num_stored])

In [ ]:
#unzoomed figure
print('first',graph_points[grange],'plots')

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)

plt.plot(graph_points[0:(all_presented+1)],OM_learn_Vacc[0:(all_presented+1)],color='darkblue')
plt.plot(graph_points[all_presented:grange],OM_learn_Vacc[all_presented:grange],color='cornflowerblue', label='OM')
plt.plot(graph_points[0:grange],tf_learn_Vacc[0:grange],color='orange', label='TF')



plt.legend(loc='lower right')
plt.ylabel('Validation Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.ylim([0.4,1.0])

plt.title("OM vs TF {} {} {} trained total".format(dataset_name,MODEL,graph_points[grange]))
#plt.xticks(np.arange(0,graph_points[num_stored]+1,int(graph_points[num_stored]/10)))  # Set label locations.
plt.xticks(np.arange(0,graph_points[grange]+1,int(graph_points[grange]/10)))  # Set label locations.



plt.xlabel('Batch Number')
plt.show()
print('Labels  ',np.array(labels_presented)[0:grange].T)